In [1]:
import spacy 
#from spacy.lang.en import English
#from spacy.pipeline import EntityRuler
import json
import random

In [ ]:
#don't mess with the codeblock box
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
    return(patterns)
patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
print(patterns)

In [ ]:
#don't mess with this codeblock box 
# #TX NER TRAINING SET 
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                #print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
        return(patterns)

def generate_rules(patterns):
    nlp = English()
    #ruler = EntityRuler(nlp)
    ruler = nlp.add_pipe('entity_ruler', config={"validate": True})
    ruler.add_patterns(patterns) 
    nlp.to_disk("tx_trained_ner")


def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) >0:
        results = [text, {"entities": entities}]
        print(results)
    return (results)

patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
generate_rules(patterns)
#print(patterns)


nlp = spacy.load("tx_trained_ner")
ie_data={}
with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:28]
    hits=[]
    #cleaning up the lyric text. making it easier to read for the program
    for segment in segments:
        #segment = segment.strip()
        #segment = segment.replace()
        results = test_model(nlp, segment)
        #print(results)
        for result in results:
            hits.append(result)
    ie_data[segment] = hits
        #if results != None:
            #TRAIN_DATA.append(results)
    #print (ie_data)

#save_data("../Corridos/data/TxMunicipality_training_set.json", ie_data)
  

In [19]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       json.dump(data, f, indent=4)
       #print(data)

def train_spacy(data, iterations): #passing training data/set and the number/generation of the training process. 30 is good. long time!!!!
    #print(data)
    TRAIN_DATA = data
    #print(TRAIN_DATA)
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        #if there is no ner in the pipe, create a pipe
        ner = nlp.create_pipe("ner")
        nlp.add_pipe('ner', last=True)
    #add labels
    for _, annotations in TRAIN_DATA():
        print(annotations)
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipe(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration "+ str(itn))
            random.shuffle(TRAIN_DATA) #shuffle helps the program to learn not memorize order. 
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2, #to help avoid over fitting which is a common practice in machine learning. 
                            sgd= optimizer,
                            losses=losses
                            )
            print(losses)  
    return(nlp)

TRAIN_DATA = load_data("../Corridos/data/TxMunicipality_training_set.json")
nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("tx_ner_model")

TypeError: 'dict' object is not callable